In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score

In [10]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Load the dataset
df = pd.read_csv(r"C:\Users\sonur\OneDrive\Desktop\Car_Dheko\Merged_car\EDA_Cars.csv")

# Clean numeric columns
numeric_cols = ['kilometers_driven', 'registration_year', 'price', 'seats', 'kms_driven', 'bhp', 'rpm']
for col in numeric_cols:
    if df[col].dtype == 'object':
        df[col] = pd.to_numeric(df[col].str.replace(',', '').str.replace('₹', '').str.strip(), errors='coerce')

# Handle categorical variables (if 'state' is present, convert it to category)
if 'state' in df.columns:
    df['state'] = df['state'].astype('category').cat.codes

# One-Hot Encoding for other categorical features
categorical_cols = df.select_dtypes(include=['object', 'category']).columns.tolist()
if 'state' in categorical_cols:
    categorical_cols.remove('state')  # Exclude 'state' if it's already encoded

df = pd.get_dummies(df, columns=categorical_cols, drop_first=True)

# Check for missing values in the DataFrame
print("Missing values in DataFrame:\n", df.isnull().sum())

# Option 1: Drop rows with NaN values in the target column
df = df.dropna(subset=['price'])

# Option 2: (Uncomment if you prefer to fill missing values)
# df['price'].fillna(df['price'].median(), inplace=True)  # Fill NaN with median value

# Split the data into features and target
X = df.drop('price', axis=1)
y = df['price']

# Check for any remaining NaN values in y
if y.isnull().any():
    print("NaN values found in target variable y.")

# Train-Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Model Selection and Training
model = RandomForestRegressor(random_state=42)

# Hyperparameter Tuning using Grid Search
param_grid = {
    'n_estimators': [100, 200],
    'max_features': ['auto', 'sqrt'],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
}

grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=5, n_jobs=-1, verbose=2, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)

# Best parameters
print("Best Parameters:\n", grid_search.best_params_)

# Evaluate the model
best_model = grid_search.best_estimator_
y_pred = best_model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

# Save the modified DataFrame to a new CSV file
df.to_csv(r"C:\Users\sonur\OneDrive\Desktop\Car_Dheko\Merged_car\Car_ModelDev.csv", index=False)


Missing values in DataFrame:
 kilometers_driven                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         

C:\Users\sonur\AppData\Roaming\Python\Python312\site-packages\sklearn\model_selection\_validation.py:540: FitFailedWarning: 
120 fits failed out of a total of 240.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
95 fits failed with the following error:
Traceback (most recent call last):
  File "C:\Users\sonur\AppData\Roaming\Python\Python312\site-packages\sklearn\model_selection\_validation.py", line 888, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "C:\Users\sonur\AppData\Roaming\Python\Python312\site-packages\sklearn\base.py", line 1466, in wrapper
    estimator._validate_params()
  File "C:\Users\sonur\AppData\Roaming\Python\Python312\site-packages\sklearn\base.py", line 666, in _validate_params
    valid

Best Parameters:
 {'max_depth': None, 'max_features': 'sqrt', 'min_samples_split': 2, 'n_estimators': 100}
Mean Squared Error: 105605558.55959584


In [11]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import LabelEncoder

# Load the dataset
file_path = r'C:\Users\sonur\OneDrive\Desktop\Car_Dheko\Merged_car\merged_car.csv'
df = pd.read_csv(file_path)

# Display missing values
print("Missing values in DataFrame:")
print(df.isnull().sum())

# Handling missing values (example strategy)
df['price'] = df['price'].replace({'₹ ': '', ' Lakh': ''}, regex=True).astype(float)
df['registration_year'] = pd.to_datetime(df['registration_year'], errors='coerce').dt.year
df['mileage'] = df['mileage'].replace({' kmpl': ''}, regex=True).astype(float)

# Drop rows with missing target variable 'price'
df.dropna(subset=['price'], inplace=True)

# Fill missing values in 'seats' with the mean
df['seats'].fillna(df['seats'].mean(), inplace=True)

# Convert categorical columns using Label Encoding
label_cols = ['fuel_type', 'body_type', 'transmission', 'owner', 'oem', 'model', 'variant']
for col in label_cols:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col].astype(str))

# Define feature and target variables
X = df.drop(columns=['price'])  # Features
y = df['price']  # Target variable

# Train-test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Set up the Random Forest Regressor and Grid Search for hyperparameter tuning
rf = RandomForestRegressor()

# Hyperparameter grid
param_grid = {
    'n_estimators': [50, 100, 150],
    'max_features': ['sqrt', 'log2', None],
    'min_samples_split': [2, 5, 10],
    'max_depth': [None, 10, 20, 30],
}

# Grid search
grid_search = GridSearchCV(estimator=rf, param_grid=param_grid, cv=5, n_jobs=-1, verbose=1)
grid_search.fit(X_train, y_train)

# Best parameters
print("Best parameters from grid search:")
print(grid_search.best_params_)

# Make predictions
y_pred = grid_search.predict(X_test)

# Evaluate model performance
mse = mean_squared_error(y_test, y_pred)
print(f"Mean Squared Error: {mse}")

# Optional: Save the model if needed
import joblib
joblib.dump(grid_search.best_estimator_, 'random_forest_model.pkl')


Missing values in DataFrame:
City                         0
car_links                    0
fuel_type                    0
body_type                    4
kilometers_driven            0
transmission                 0
owner                        0
oem                          0
model                        0
year                         0
variant                      0
price                        0
registration_year           37
insurance_validity           7
fuel_type_overview           0
seats                        7
kms_driven                   2
rto                        952
comfort_convenience         65
interior_features           76
exterior_features           90
safety_features             97
entertainment_features    1078
mileage                   8431
engine                    8431
max_power                   65
torque                    8431
wheel_size                8431
bhp                       2354
rpm                       2354
consolidated_data            0
dtype: int

ValueError: could not convert string to float: '58,000 '